# LHS is not available on TS machines and will be performed elsewhere

In [6]:
import numpy as np
import pandas as pd
from scipy.stats import qmc
from pathlib import Path
import sys

# --------------------------
# Setup for parameter sampling
# --------------------------
params = ['E_GPa', 'PR', 'SH_MPa/km', 'Sh_MPa/km', 'Sv_MPa/km', 'SH_azi_deg']
l_bounds = [15, 0.2, 25 * 0.9, 14.6 * 0.9, 22.7 * 0.9, 290]
u_bounds = [25, 0.4, 25 * 1.1, 14.6 * 1.1, 22.7 * 1.1, 310]

# --------------------------
# Setup for poro/permx file selection
# --------------------------
data_dir = Path('..') / 'data' / 'properties'

if not data_dir.exists():
    print(f"Error: Directory '{data_dir}' does not exist.")
    sys.exit(1)

poro_files = sorted([f for f in data_dir.iterdir() if "PORO" in f.name])
permx_files = sorted([f for f in data_dir.iterdir() if "PERMX" in f.name])

if not poro_files or not permx_files:
    print("Error: PORO or PERMX files not found in data directory.")
    sys.exit(1)

# --------------------------
# Latin Hypercube Sampling
# --------------------------
num_samples = 3
# Total dimensions = 6 physical parameters + 2 file selectors
sampler = qmc.LatinHypercube(d=len(params) + 2)
sample = sampler.random(n=num_samples)

# Scale physical parameters
sample_params = qmc.scale(sample[:, :len(params)], l_bounds, u_bounds)
df_params = pd.DataFrame(data=np.round(sample_params, 2), columns=params)

# Scale poro/permx indices
poro_indices = (sample[:, -2] * len(poro_files)).astype(int)
permx_indices = (sample[:, -1] * len(permx_files)).astype(int)

# Ensure indices are in bounds
poro_indices = np.clip(poro_indices, 0, len(poro_files) - 1)
permx_indices = np.clip(permx_indices, 0, len(permx_files) - 1)

# Add poro/permx file paths
df_params["poro_file"] = [str(poro_files[i]) for i in poro_indices]
df_params["permx_file"] = [str(permx_files[i]) for i in permx_indices]

# calculate stress state params for CMG dat file
beta = 300-90 #rotate from SH to x axis
cos_2beta = np.cos(np.radians(2*beta))
sin_2beta = np.sin(np.radians(2*beta))
df_params['sigma_x']=(df_params['SH_MPa/km']+df_params['Sh_MPa/km'])/2 + (df_params['SH_MPa/km']-df_params['Sh_MPa/km'])/2*cos_2beta
df_params['sigma_y']=(df_params['SH_MPa/km']+df_params['Sh_MPa/km'])/2 - (df_params['SH_MPa/km']-df_params['Sh_MPa/km'])/2*cos_2beta
df_params['tau_xy'] = (df_params['SH_MPa/km']-df_params['Sh_MPa/km'])/2*sin_2beta

# --------------------------
# Final result
# --------------------------
df_params


,E_GPa,PR,SH_MPa/km,Sh_MPa/km,Sv_MPa/km,SH_azi_deg,poro_file,permx_file,sigma_x,sigma_y,tau_xy
0,19.92,0.34,22.59,14.24,21.71,300.76,../data/properties/JD_BASECASE_3_PORO.dat,../data/properties/JD_BASECASE_2_PERMX.dat,20.5025,16.3275,3.615656
1,23.59,0.31,24.82,13.60,24.37,305.07,../data/properties/JD_BASECASE_5_PORO.dat,../data/properties/JD_BASECASE_6_PERMX.dat,22.0150,16.4050,4.858403
2,17.76,0.22,26.73,15.60,22.56,294.96,../data/properties/JD_BASECASE_1_PORO.dat,../data/properties/JD_BASECASE_3_PERMX.dat,23.9475,18.3825,4.819431
